<a href="https://colab.research.google.com/github/poporubeus/Chess_network/blob/main/Data_scraping_chess_con_modifica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Importing**
Importare le varie librerie

In [ ]:
import pandas as pd
import requests
from requests import Session
from bs4 import BeautifulSoup
from urllib.parse import urlparse

Funzione che prende link alla pagina principale dei tornei e restituisce in output lista con tutte le pagine di tutti gli anni

In [ ]:
session = Session()


# tournaments home https://www.365chess.com/tournaments.php


def get_tournaments_pages(url, year):
    lista_link=[]
    
    response = session.get(
        url=url,
        headers={
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
            'Accept-Encoding': 'gzip, deflate, br',
            'Accept-Language': 'en-GB,en-US;q=0.9,en;q=0.8,it;q=0.7',
            'Connection': 'keep-alive',
            'Host': 'www.365chess.com',
            'sec-ch-ua': '"Google Chrome";v="111", "Not(A:Brand";v="8", "Chromium";v="111"',
            'sec-ch-ua-mobile': '?0',
            'sec-ch-ua-platform': '"macOS"',
            'Sec-Fetch-Dest': 'document',
            'Sec-Fetch-Mode': 'navigate',
            'Sec-Fetch-Site': 'none',
            'Sec-Fetch-User': '?1',
            'Upgrade-Insecure-Requests': '1',
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36',
        }
    )

    if response:
        #print("Get in Tournaments Page")
        base_url = f'https://{urlparse(response.url).hostname}/'
        soup = BeautifulSoup(response.text, 'html.parser')
        pages = soup.find('table', {'class': 'pag'})
        if pages and pages.td and pages.td.findAll('a'):
            # collect all pages

            # adding first page
            pages_dict = {
                # number: url
                '1': response.url  # or tournament_url
            }
            for a in [a for a in pages.td.findAll('a') if a.attrs and a.attrs.get('href') and not a.text == 'Next']:
                # number: url
                pages_dict[a.text] = base_url + a.attrs.get('href')

            last_page_number = int(list(pages_dict.keys())[-1])
            if last_page_number != len(pages_dict):
                # fix hidden pages
                pages_dict = {'1': f'{base_url}?p=1&tname=&year={year}&country=&avgelo='}
                count_start = 0
                for num in range(2, last_page_number + 1):
                    count_start += 60
                    pages_dict[str(num)] = f'{base_url}tournaments.php?p=1&tname=&year={year}&country=&avgelo=&start={count_start}'
            
         

            # print all pages
            for page_number, page_url in pages_dict.items():
                lista_link.append(page_url)
               
    
    return(lista_link)


Qui viene fatto un loop sugli anni e si costruisce una lista di link. Vorrei in qualche maniera però averli raggruppati per anni. Non so fino a quanto possa essere un problema ma bon, vi avviso.

**L'unica cosa che devi fare qui** è settare **start= anno di inizio**,
**finish = anno di fine** ...elementare **FAV**... 

In [ ]:
start = 1988
finish = 1990
years = list(range(start,finish))
lista_pagine=[]
#questo è il loop sugli anni che salva in una lista
for selected_year in years:
  lista_pagine.append(get_tournaments_pages(f'https://www.365chess.com/tournaments.php?year={selected_year}', selected_year)) 
#questo linearifica la lista di liste in una lista di oggetti
lista_pagine_linear = [item for sublist in lista_pagine for item in sublist]


Ora, dagli url per gli anni, cerco di ottenere gli url dei tornei

In [ ]:
def get_tournaments(url):
  lista2=[]
  soup = BeautifulSoup(requests.get(str(url)).text, 'html.parser')
  pages = soup.find('table', {'class': 'hoverable stable'})
  for a in pages.find_all('a', href=True):
    lista2.append(a['href'])
  return(lista2)

qui provo che funge e funge. Manca capire come muoversi tra le pagine

In [ ]:
linkissimo=[]

link_ke_non_funzia = 'https://www.365chess.com/?p=1&tname=&year=1989&country=&avgelo='
for link in lista_pagine_linear:
    print(link)
    if (link == link_ke_non_funzia):
     # link == 'https://www.365chess.com/?p=1&tname=&year=2019&country=&avgelo='):
      pass
    else:
      linkissimo.append(get_tournaments(link))
print(linkissimo)

https://www.365chess.com/tournaments.php?year=1988
https://www.365chess.com/tournaments.php?p=1&tname=&year=1988&country=&avgelo=&start=60
https://www.365chess.com/tournaments.php?p=1&tname=&year=1988&country=&avgelo=&start=120
https://www.365chess.com/tournaments.php?p=1&tname=&year=1988&country=&avgelo=&start=180
https://www.365chess.com/tournaments.php?p=1&tname=&year=1988&country=&avgelo=&start=240
https://www.365chess.com/tournaments.php?p=1&tname=&year=1988&country=&avgelo=&start=300
https://www.365chess.com/tournaments.php?p=1&tname=&year=1988&country=&avgelo=&start=360
https://www.365chess.com/tournaments.php?p=1&tname=&year=1988&country=&avgelo=&start=420
https://www.365chess.com/tournaments.php?p=1&tname=&year=1988&country=&avgelo=&start=480
https://www.365chess.com/?p=1&tname=&year=1989&country=&avgelo=
https://www.365chess.com/tournaments.php?p=1&tname=&year=1989&country=&avgelo=&start=60
https://www.365chess.com/tournaments.php?p=1&tname=&year=1989&country=&avgelo=&start=1

In [ ]:
linkissimo

[['https://www.365chess.com/tournaments/RLPF-ch_U13_1988/20773',
  'https://www.365chess.com/tournaments/National_op_1988/20745',
  'https://www.365chess.com/tournaments/Nesselwang_1988/20746',
  'https://www.365chess.com/tournaments/New_York_m_1988/20747',
  'https://www.365chess.com/tournaments/New_York_op-B_1988/20748',
  'https://www.365chess.com/tournaments/New_York_sim_1988/20749',
  'https://www.365chess.com/tournaments/Norrkoeping_1988/20750',
  'https://www.365chess.com/tournaments/Nuremberg_op_1988/20751',
  'https://www.365chess.com/tournaments/Oakham_House_1988/20752',
  'https://www.365chess.com/tournaments/Nassau-ch_1988/20744',
  'https://www.365chess.com/tournaments/Napoli_op_1988/20743',
  'https://www.365chess.com/tournaments/Muenster_Elo_1988/20735',
  'https://www.365chess.com/tournaments/Muenster_op_1988/20736',
  'https://www.365chess.com/tournaments/Myslenice_1988/20737',
  'https://www.365chess.com/tournaments/NOR-ch_1988/20738',
  'https://www.365chess.com/tour

#### Qui interviene FAV aggiungendo 2 robette giusto per capire come funziona una lista di liste, siccome non gli è ancora chiaro...

In [ ]:
col = []
for i in range(len(linkissimo)):
  col.append(linkissimo[i][:])

In [ ]:
col

[['https://www.365chess.com/tournaments/RLPF-ch_U13_1988/20773',
  'https://www.365chess.com/tournaments/National_op_1988/20745',
  'https://www.365chess.com/tournaments/Nesselwang_1988/20746',
  'https://www.365chess.com/tournaments/New_York_m_1988/20747',
  'https://www.365chess.com/tournaments/New_York_op-B_1988/20748',
  'https://www.365chess.com/tournaments/New_York_sim_1988/20749',
  'https://www.365chess.com/tournaments/Norrkoeping_1988/20750',
  'https://www.365chess.com/tournaments/Nuremberg_op_1988/20751',
  'https://www.365chess.com/tournaments/Oakham_House_1988/20752',
  'https://www.365chess.com/tournaments/Nassau-ch_1988/20744',
  'https://www.365chess.com/tournaments/Napoli_op_1988/20743',
  'https://www.365chess.com/tournaments/Muenster_Elo_1988/20735',
  'https://www.365chess.com/tournaments/Muenster_op_1988/20736',
  'https://www.365chess.com/tournaments/Myslenice_1988/20737',
  'https://www.365chess.com/tournaments/NOR-ch_1988/20738',
  'https://www.365chess.com/tour

In [ ]:
for i in range(len(col)):
  print(len(col[i]))

#### questi sono i link ai tornei nei 2 anni (2010,2011) per pagina (2010 ha 7 pagine mentre 2011 ne ha 8)

60
60
60
60
60
60
60
60
44
60
60
60
60
60
60
60
60
60
40


In [ ]:
len(col)

### Assicurati che Python faccia scraping di 19 pagine (dalla 0 alla 18)

19

#### *Proviamo a riapplicare il codice del tizio per accedere ai tornei dentro ciascun anno*



In [ ]:
session = Session()


# tournaments home https://www.365chess.com/tournaments.php


def get_torneo(url):
    lista_link=[]
    
    response = session.get(
        url=url,
        headers={
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
            'Accept-Encoding': 'gzip, deflate, br',
            'Accept-Language': 'en-GB,en-US;q=0.9,en;q=0.8,it;q=0.7',
            'Connection': 'keep-alive',
            'Host': 'www.365chess.com',
            'sec-ch-ua': '"Google Chrome";v="111", "Not(A:Brand";v="8", "Chromium";v="111"',
            'sec-ch-ua-mobile': '?0',
            'sec-ch-ua-platform': '"macOS"',
            'Sec-Fetch-Dest': 'document',
            'Sec-Fetch-Mode': 'navigate',
            'Sec-Fetch-Site': 'none',
            'Sec-Fetch-User': '?1',
            'Upgrade-Insecure-Requests': '1',
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36',
        }
    )

    if response:
        #print("Get in Tournaments Page")
        base_url = f'https://{urlparse(response.url).hostname}/'
        soup = BeautifulSoup(response.text, 'html.parser')
        pages = soup.find('table', {'class': 'pag'})
        #print(pages.td.findAll('a'))
        if pages.td.findAll('a') == []:
          
          return [url]
        else: 
            # collect all pages

            # adding first page
            pages_dict = {
                # number: url
                '1': response.url  # or tournament_url
            }
            for a in [a for a in pages.td.findAll('a') if a.attrs and a.attrs.get('href') and not a.text == 'Next']:
                # number: url
                pages_dict[a.text] = base_url + a.attrs.get('href')

            last_page_number = int(list(pages_dict.keys())[-1])
            if last_page_number != len(pages_dict):
                # fix hidden pages
                pages_dict = {'1': f'{url}/?p=1&start='}
                count_start = 0
                for num in range(2, last_page_number + 1):
                    count_start += 100
                    pages_dict[str(num)] = f'{url}/?p=1&start={count_start}'
                    
            
            
            # print all pages
            for page_number, page_url in pages_dict.items():
                lista_link.append(page_url)
           
            return lista_link
   

###**CARTELLA** da caricare su drive

In [ ]:
import os

path = '/content'
folder = '/chess_games'

os.mkdir(path+folder)

#**Purtroppo qui subentra un problema fastidioso:**
###Python aggiunge due volte lo stesso url */https://www.365chess.com* con lo / davanti. Questo perchè il codice del mio amico non gestisce bene il dictonary, ovvero aggiunge 2 volte lo stesso url in un for, che è dentro un if, nel quale viene aggiunto la prima volta. Come risolvere questo prob?
### **Metodo ZAPPATORE:**
###Una volta individuata questa stringa la si toglie a mano e si mette tutto dentro un loop con check sulle pagine (quando vedi che il link del torneo ha tante pagine e non solo una, allora applica questo)!

###Come si vede, l'url compare 2 volte qui sopra 🔝

In [ ]:
import time

#**LEGGI BENE PRIMA!**

Ci sono degli anni che mostrano dei problemi...ho trovato errori del tipo "Invalid ascii code"...insomma roba strana.
Supponi dunque che l'errore si presenti al torneo X.
Qui ho semplicemente messo l'istruzione "**df.to_csv('/content/chess_games/chess_data_8890.csv', index=False)**" all'interno del primo **for** (vedi indentazione). In questo modo ogni torneo viene già salvato, cosicché se capita un errore al torneo X, noi abbiamo già gli X-1 links.

Se questo errore non si presenta allora è possibile mettere fuori dal **for** il comando (che in realtà c'è già ma è commentato)

In [ ]:
string_to_check = '/?p=1&start='
string_2_delete = '/https://www.365chess.com'


df = pd.DataFrame()
for i in range(len(col)):
  for j in range(len(col[i])):
    print('Pag',i,', T:',j)
    time.sleep(2)
    pagine = get_torneo(str(col[i][j]))
    
#list_of_final_urls = []
    #for url in pagine:
    time.sleep(2)
    df = pd.concat([df,pd.DataFrame(pd.read_html(pagine[0])[2])])
    for k in range(1,len(pagine)):
      if string_to_check in pagine[k]:
        pagine[k]=pagine[k].replace(string_2_delete,'')
      print('Tpag:',k)
      time.sleep(2)
      df = pd.concat([df,pd.DataFrame(pd.read_html(pagine[k])[1])])
  df.to_csv('/content/chess_games/chess_data_8890.csv', index=False)
      
    
#df.to_csv('/content/chess_games/chess_data_8890.csv', index=False)

Pag 0 , T: 0
Pag 0 , T: 1
Tpag: 1
Pag 0 , T: 2
Pag 0 , T: 3
Pag 0 , T: 4
Pag 0 , T: 5
Pag 0 , T: 6
Pag 0 , T: 7
Tpag: 1
Pag 0 , T: 8
Pag 0 , T: 9
Tpag: 1
Pag 0 , T: 10
Pag 0 , T: 11
Pag 0 , T: 12
Pag 0 , T: 13
Pag 0 , T: 14
Pag 0 , T: 15
Tpag: 1
Tpag: 2
Tpag: 3
Pag 0 , T: 16
Pag 0 , T: 17
Pag 0 , T: 18
Pag 0 , T: 19
Pag 0 , T: 20
Pag 0 , T: 21
Pag 0 , T: 22
Pag 0 , T: 23
Pag 0 , T: 24
Pag 0 , T: 25
Pag 0 , T: 26
Pag 0 , T: 27
Pag 0 , T: 28
Pag 0 , T: 29
Pag 0 , T: 30
Pag 0 , T: 31
Pag 0 , T: 32
Tpag: 1
Pag 0 , T: 33
Pag 0 , T: 34
Pag 0 , T: 35
Pag 0 , T: 36
Pag 0 , T: 37
Pag 0 , T: 38
Pag 0 , T: 39
Pag 0 , T: 40
Pag 0 , T: 41
Pag 0 , T: 42
Pag 0 , T: 43
Pag 0 , T: 44
Pag 0 , T: 45
Pag 0 , T: 46
Pag 0 , T: 47
Pag 0 , T: 48
Pag 0 , T: 49
Pag 0 , T: 50
Pag 0 , T: 51
Pag 0 , T: 52
Pag 0 , T: 53
Pag 0 , T: 54
Pag 0 , T: 55
Pag 0 , T: 56
Pag 0 , T: 57
Tpag: 1
Pag 0 , T: 58
Pag 0 , T: 59
Pag 1 , T: 0
Tpag: 1
Tpag: 2
Pag 1 , T: 1
Tpag: 1
Tpag: 2
Pag 1 , T: 2
Pag 1 , T: 3
Pag 1 , T: 4
Pag 1 , T

In [ ]:
col[15][48]

Lo sleep non fa crashare il sito.
Fare un save dopo ogni concat "write_csv()" su google drive e anche i link scaricati, cosicchè prima di fare read_html guardiamo se il link è dentro quelli già salvati, in quel caso, skippa.
Magari salvare ogni tabella corrispondente all'url in un file diverso .csv con il nome dell'url. Dopo fare una concatenazione alla fine di tutto a parte dopo una lettura